In [3]:
# import
import pandas as pd
import numpy as np
import selenium
import sqlalchemy
from urllib import parse
import requests
from bs4 import BeautifulSoup as BS
import os
import json
from tqdm import tqdm
import pickle
from geopy.geocoders import Nominatim
import subprocess
import time
import lxml

# custom python lib
import src.naver_geocoding_api as nga
import src.evchargestations as evcs
import src.connect_to_db as cdb


In [198]:
with open("./keys/api_key.json", "r") as f:
    key_json = json.load(f)

In [239]:
def get_restarea_info()->pd.DataFrame:
    with requests.Session() as session:
        try:
            restarea_api_url_template = "https://data.ex.co.kr/openapi/locationinfo/locationinfoRest?{}{}{}{}"
            restarea_api_key = f"key={key_json['RESTAREA_API_KEY']}"
            restarea_api_type = f"&type=json"
            restarea_api_rownum = "&numOfRows=10"
            restarea_api_pageNo = "&pageNo={}"
            restarea_url = restarea_api_url_template.format(restarea_api_key, restarea_api_type, restarea_api_rownum,restarea_api_pageNo.format(1))
            
            init_req = session.get(restarea_url)
            restarea_df = pd.DataFrame(init_req.json()['list'])
            
            assert init_req.status_code==200, nga.log(f"requests status code exited with {init_req.status_code}")
            with open("./results/rest_areas.json", "w", encoding='utf-8') as f:
                json.dump(init_req.json(), f, ensure_ascii=False)
            
            page_size = init_req.json()['pageSize']
            
            for pageNo in tqdm(range(2, page_size+1)):
                restarea_url_total = restarea_api_url_template.format(restarea_api_key, restarea_api_type, restarea_api_rownum,restarea_api_pageNo.format(pageNo))
                concat_req = session.get(restarea_url_total)
                
                assert concat_req.status_code==200, nga.log(f"requests status code exited with {concat_req.status_code}")
                with open("./results/rest_areas.json", "w", encoding='utf-8') as f:
                    json.dump(concat_req.json(), f, ensure_ascii=False)
                concat_df = pd.DataFrame(concat_req.json()['list'])
                restarea_df = pd.concat([restarea_df, concat_df])
            return restarea_df
        except Exception as e:
            nga.log(f"Exception: {e}")
            return None
restarea_df = get_restarea_info()

100%|██████████| 20/20 [00:02<00:00,  9.00it/s]


In [245]:
restarea_df.head(5)

,unitName,pageNo,numOfRows,unitCode,stdRestCd,routeNo,routeName,xValue,yValue,serviceAreaCode
0,서울만남(부산)휴게소,None,None,001,000001,0010,경부선,127.042514,37.459939,A00001
1,죽전(서울)휴게소,None,None,002,000003,0010,경부선,127.104397,37.332583,A00002
2,기흥(부산)휴게소,None,None,003,000005,0010,경부선,127.105190,37.235090,A00003
3,안성(서울)휴게소,None,None,004,000007,0010,경부선,127.131800,37.076443,A00004
4,안성(부산)휴게소,None,None,005,000009,0010,경부선,127.145006,37.013628,A00005


In [258]:
import openpyxl
with open("./data/cs_list.xls", "r") as f:
    cs_list_df = pd.read_excel(f, engine="openpyxl")

BadZipFile: File is not a zip file

In [244]:
with open("./results/highway_rest_areas_df.pkl", "wb") as f:
    pickle.dump(restarea_df, f)

In [4]:
with open("./data/gas_station_final.pkl", "rb") as f:
    gas_station_df = pd.read_pickle(f)

In [5]:
gas_station_df.shape

(10766, 6)

In [6]:
testdf = gas_station_df.head(5).copy()

In [9]:
testdf.head(5)

,sido,sigungu,station_name,address,company,phone_number,coordinates
0,서울특별시,강남구,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,02-445-6870,"[127.1187331, 37.4672785]"
1,서울특별시,강남구,HD현대오일뱅크㈜직영 산성셀프주유소,서울 강남구 헌릉로 730,HD현대오일뱅크,02-2226-4963,"[127.1190231, 37.4665975]"
2,서울특별시,강남구,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,02-445-5841,"[127.1040159, 37.4835419]"
3,서울특별시,강남구,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,02-459-3434,"[127.1068180, 37.4749811]"
4,서울특별시,강남구,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,02-3462-5100,"[127.0395832, 37.4858704]"


In [8]:
tqdm.pandas()
testdf['coordinates'] = testdf['address'].progress_apply(lambda x: nga.geocoding(x))

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


In [7]:
tqdm.pandas()
gas_station_df['coordinates'] = gas_station_df['address'].progress_apply(lambda x: nga.geocoding(x))
with open("./data/gas_station_final.pkl", "wb") as f:
    pickle.dump(gas_station_df, f)

100%|██████████| 10766/10766 [1:32:12<00:00,  1.95it/s]  


In [8]:
with open("./data/gas_station_final.pkl", "rb") as f:
    gas_data = pickle.load(f)

In [11]:
gas_station_df.head(5)

,sido,sigungu,station_name,address,company,phone_number,coordinates
0,서울특별시,강남구,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,02-445-6870,"[0, 0]"
1,서울특별시,강남구,HD현대오일뱅크㈜직영 산성셀프주유소,서울 강남구 헌릉로 730,HD현대오일뱅크,02-2226-4963,"[0, 0]"
2,서울특별시,강남구,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,02-445-5841,"[0, 0]"
3,서울특별시,강남구,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,02-459-3434,"[0, 0]"
4,서울특별시,강남구,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,02-3462-5100,"[0, 0]"


In [10]:
gas_data[gas_data['coordinates'].apply(lambda x:x[0]==0 and x[1]==0)].shape

(10766, 7)

In [230]:
testdf[testdf['coordinates'].apply(lambda x:x[0]==0 and x[1]==0)]

,sido,sigungu,station_name,address,company,phone_number,coordinates
